### Install dependencies

In [ ]:
!pip install transformers
!pip install torch

## Import data

In [ ]:
import pandas as pd
import bz2

with bz2.open('/content/drive/MyDrive/Universitet/Thesis/NLP code/speeches-201718.json.bz2') as source:
    speeches_201718 = pd.read_json(source)

with bz2.open('/content/drive/MyDrive/Universitet/Thesis/NLP code/speeches-201819.json.bz2') as source:
    speeches_201819 = pd.read_json(source)

speeches = pd.concat([speeches_201718, speeches_201819])

### Preprocess data


In [ ]:
from sklearn.model_selection import train_test_split

party_to_int = {
    'S': 0,
    'M': 1,
    'MP': 2,
    'SD': 3,
    'V': 4,
    'C': 5,
    'KD': 6,
    'L': 7,
}

def speeches_split(speeches, max_text_length=1000):
  texts = []
  labels = []
  for _, row in speeches.iterrows():
    words_arr = row["words"].split(" ")
    words_arr = words_arr[-max_text_length:]
    words = ' '.join(map(str, words_arr)) 
    texts.append(words)
    party = row["party"]
    party_int = party_to_int[party]
    labels.append(party_int)
  return texts, labels

train_texts, train_labels = speeches_split(speeches)

# Split data set into 80/20
train_texts, test_texts, train_labels, test_labels = train_test_split(train_texts, train_labels, test_size=.2)


## Import pre-trained Swedish bert and tokenize data

In [ ]:
from transformers import AutoModel,AutoTokenizer,TFAutoModel

tokenizer = AutoTokenizer.from_pretrained('KB/bert-base-swedish-cased')
model = AutoModel.from_pretrained('KB/bert-base-swedish-cased')

train_encodings = tokenizer(train_texts, truncation=True, max_length=256, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, max_length=256, padding=True)

## Plot data

In [ ]:
import matplotlib.pyplot as plt

# Distribution of speeches over the parties
fig, axes = plt.subplots(nrows=1,ncols=2)
speeches['party'].value_counts().plot(ax=axes[0],subplots=True, kind='bar', color=["red", "blue", "limegreen", "yellow", "darkred", "seagreen", "darkblue", "lightskyblue"], x="Party", figsize=(5, 3))
axes[0].title.set_text('Class distribution')
axes[0].set_xlabel('Party')
axes[0].set_ylabel('Number of speeches')

# Token lengths for all speeches
# for box plot
len_tokens = []
for speech in speeches["words"]:
  t = tokenizer(speech)
  len_tokens.append(len(t["input_ids"])-2)

fig.subplots_adjust(wspace=0.5)
fig.set_figheight(5)
fig.set_figwidth(10)

data = [len_tokens]
axes[1].set_title('Statistical analysis')
axes[1].boxplot(data, showfliers=False, labels=["Speeches"])
axes[1].set_ylabel("Number of tokens")
axes[1].set_xlabel("Data set")

plt.show()

In [ ]:
# Save fig
from google.colab import files
fig.savefig("abc.png")
files.download("abc.png") 


## Setup for pytorch


In [ ]:
import torch

class SpeechDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SpeechDataset(train_encodings, train_labels)
test_dataset = SpeechDataset(test_encodings, test_labels)

## Adapt pre-trained BERT
Add a dropout layer, and a linear layer on top of the pooled BERT output.
Calculate cross entropy loss.

In [ ]:
from torch import nn

class CustomBERTModel(nn.Module):
    def __init__(self):
        super(CustomBERTModel, self).__init__()
        self.num_labels = 8

        self.bert = AutoModel.from_pretrained('KB/bert-base-swedish-cased')
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(768, 8)

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        r"""
        labels (:obj:`torch.LongTensor` of shape :obj:`(batch_size,)`, `optional`):
            Labels for computing the sequence classification/regression loss. Indices should be in :obj:`[0, ...,
            config.num_labels - 1]`. If :obj:`config.num_labels == 1` a regression loss is computed (Mean-Square loss),
            If :obj:`config.num_labels > 1` a classification loss is computed (Cross-Entropy).
        """

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        loss_fct = nn.CrossEntropyLoss()
        loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
        output = (logits,) + outputs[2:]
        return ((loss,) + output) if loss is not None else output


## Trainer setup

In [ ]:
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import classification_report

model = CustomBERTModel()
model = model.cuda()
class_reports = []

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    cr = classification_report(labels, preds)
    print(cr)
    class_reports.append(cr)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }
  
epochs = 10
batch_size = 8
warm_steps = int(len(train_texts)*0.1/batch_size*epochs)

training_args = TrainingArguments(
    output_dir='./results',                      # output directory
    num_train_epochs=epochs,                         # total number of training epochs
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,      # batch size per device during training
    per_device_eval_batch_size=batch_size*2,     # batch size for evaluation
    warmup_steps=warm_steps,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                           # strength of weight decay
    logging_dir='./logs',                        # directory for storing logs
    logging_steps=10,
    save_steps=1082,
)

trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=test_dataset,             # evaluation dataset
    compute_metrics=compute_metrics
)

## Train

In [ ]:
trainer.train()

In [ ]:
# Save model
torch.save(model.state_dict(), './models.pth')

## Check logs on tensorboard

In [ ]:
# Load the TensorBoard notebook extension
# Load the TensorBoard notebook extension
!kill 528
%reload_ext tensorboard
%tensorboard --logdir logs

## Pickle save class reports

In [ ]:
import pickle 
import numpy as np

# Create array of random numbers:
np_class_reports = np.array(class_reports)

filename = 'pickled_class_reports.p'
with open(filename, 'wb') as filehandler:
    pickle.dump(np_class_reports, filehandler)

In [ ]:
# Pickle read
with open(filename, 'rb') as filehandler: 
    reloaded_array = pickle.load(filehandler)

print ('Reloaded array:',reloaded_array)